# NSGA3 Project

The objective of this project is to use the NSGA3 algorithm to optimize primer set design for bacterial DNA amplification. For this purpose, a custom object called PairSet has been created that contains all forward and reverse primers and the methods to evaluate the three objectives for each pairset (Efficiency, Coverage and Variability). The goal is to maximize efficiency and coverage and minimize variability.

I am importing a cloned repository of pymoo framework where I have added print functions in the nsga3.py file for debugging purposes and show the results after the non-dominated sorting and the following operations, where some of the **non-dominated solutions are discarded**.

In [1]:
from random import randint
import numpy as np
from pairset import PairSet
from referenceset import loadFasta, ReferenceSet
from parameters import Parameters
from pymoo_framework.pymoo.optimize import minimize
from pymoo_framework.pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo_framework.pymoo.problems.functional import ElementwiseProblem
from pymoo_framework.pymoo.util.ref_dirs import get_reference_directions
from pymoo_framework.pymoo.core.sampling import Sampling
from pymoo_framework.pymoo.core.mutation import Mutation
from pymoo_framework.pymoo.core.crossover import Crossover
from pymoo_framework.pymoo.operators.selection.rnd import RandomSelection

## Read of inputs
Rep_set.fasta file contains the bacterial DNA sequences and good_pairs.fasta contains the sequences of the primers needed to amplify bacterial DNA

In [2]:
# Initialize parameters
pars = Parameters(generations = 1, pop_size = 36, n_partitions = 7, randSeed = 1, num_thread = 1) # parallelization

# Read inputs from files
print("Read reference set")
ref = ReferenceSet("rep_set.fasta", pars)

print("Read initial primer pairs")
goodPairsNames, goodPairs = loadFasta('good_pairs.fasta')

Read reference set
Read initial primer pairs


## Implementation of custom problem and operators

Minimize the negative values of efficiency and coverage to maximize efficiency and coverage.

In [3]:
# Problem
class PrimerOptimization(ElementwiseProblem):

    def __init__(self, ref, goodPairs, pars):
        super().__init__(n_var = 1, n_obj = 3, n_constr = 0)
        self.ref = ref
        self.pars = pars
        self.PrFwd = [goodPairs[i] for i in range(0, len(goodPairs), 2)]
        self.PrRev = [goodPairs[i] for i in range(1, len(goodPairs), 2)]
        

    # evaluation of efficiency, coverage and matching-bias (variability)
    # scores of each primer set
    def _evaluate(self, x, out, *args, **kwargs):
        out['F'] = np.column_stack([- x[0].getEffSc(), - x[0].getCovSc(), x[0].getVarSc()])

In [4]:
# Sampling
class MySampling(Sampling):

    # Return a sized n population of PairSet objects
    def _do(self, problem, n_samples, **kwargs):
        X = np.full((n_samples, 1), None, dtype=object) # PairSet
        n_primers = len(problem.PrFwd)
        # Create pairsets with primers in order,
        # when they run out select them randomly
        for i in range(n_samples):
            if i < n_primers:
                X[i, 0] = PairSet(problem.PrFwd[i], problem.PrRev[i], ref, pars)
            else:
                fwd = randint(0, n_primers - 1)
                rev = randint(0, n_primers - 1)
                X[i, 0] = PairSet(problem.PrFwd[fwd], problem.PrRev[rev], ref, pars)

        return X

In [5]:
# No crossover
class NoCrossover(Crossover):

    def __init__(self):
        super().__init__(n_parents = pars.pop_size, n_offsprings = pars.pop_size)
    
    # return the same population taken as input since crossover is not
    # very suitable in this problem, but performing deepcopy to create
    # the same object elsewhere in memory and not affect the original object
    def _do(self, problem, X, **kwargs):
        p_size = len(X)
        for i in range(p_size):
            X[i, 0, 0] = copyPairSet(X[i, 0, 0])

        return X

# Perform manual copy of elements because deepcopy takes so much time
def copyList(L):
    if type(L[0]) != list:
        return [i for i in L]
    else:
        return [copyList(L[i]) for i in range(len(L))]

def copyPairSet(origPSet):
    copiedPSet = PairSet.__new__(PairSet) # Empty PairSet object
    for k, v in origPSet.__dict__.items():
        if isinstance(getattr(origPSet, k), list):
            setattr(copiedPSet, k, copyList(getattr(origPSet, k)))
        else:
            setattr(copiedPSet, k, v)
    return copiedPSet

In [6]:
# Mutation
class MyMutation(Mutation):

    def __init__(self):
        super().__init__()

    def _do(self, problem, X, **kwargs):
        # Iterate over each pair set
        for i in range(len(X)):
            pSet = X[i, 0]
            fwdSet = randint(0, 1) # forward or reverse set
            p = randint(0, pSet.setLength(fwdSet) - 1) # random primer of the set
            # perform a random mutation (flip / add / trim)
            mut = randint(0,2)

            if mut == 0: # Flip random nucleotide
                n = randint(0, pSet.prLength(fwdSet, p) - 1)
                l = randint(1, 3)
                pSet.flip(fwdSet, n, p, l)

            elif mut == 1: # Add random nucleotide to head or tail
                head = randint(0, 1)
                if pSet.prLength(fwdSet, p) < pars.maxPLen:
                    l = randint(0,3)
                    pSet.add(fwdSet, head, p, l)
                else: # If not possible to add perform trim mutation
                    pSet.trim(fwdSet, head, p)

            else: # Trim nucleotide in head or tail
                head = randint(0, 1)
                if pSet.prLength(fwdSet, p) > pars.minPLen:
                    pSet.trim(fwdSet, head, p)
                else: # If not possible to trim perform add mutation
                    l = randint(0,3)
                    pSet.add(fwdSet, head, p, l)

        return X

# Run of the algorithm with 1 generation to check the initialization in more detail

In [7]:
# Problem initialization
problem = PrimerOptimization(ref, goodPairs, pars)

# Termination criteria
termination = ('n_gen', pars.n_gen)

# Reference directions for the optimization
ref_dirs = get_reference_directions('das-dennis', n_dim = 3, n_partitions = pars.partitions)

# Algorithm initialization with custom operators
algorithm = NSGA3(
    pop_size = pars.pop_size,
    ref_dirs = ref_dirs,
    sampling = MySampling(),
    selection = RandomSelection(),
    crossover = NoCrossover(),
    mutation = MyMutation(),
    eliminate_duplicates = False
)

res = minimize(
    problem = problem,
    algorithm = algorithm,
    termination = termination,
    seed = pars.seed,
    verbose = True,
    save_history = False
)


Scores of the population: 
[[ -7.21083798  -0.30701946   1.50485124]
 [ -7.39394555  -0.36474962   1.36505336]
 [ -5.83274587  -0.31423573   1.51681093]
 [ -5.87688148  -0.31379838   1.51456411]
 [ -7.13539938  -0.31226766   1.48646954]
 [ -7.98785679  -0.36868576   1.35386463]
 [ -6.17112168  -0.321452     1.49954154]
 [ -8.6410855   -0.68991909   0.70355759]
 [ -9.66666667  -0.68510824   0.69536253]
 [-10.          -0.69975946   0.6718967 ]
 [ -9.07441883  -0.68663897   0.69297983]
 [ -8.36330772  -0.78504264   0.55892714]
 [ -8.4361602   -0.61403892   0.83489132]
 [ -8.63333333  -0.69188716   0.71107467]
 [ -8.8627451   -0.68707632   0.70310844]
 [ -8.96078431  -0.70129018   0.67984835]
 [ -8.84313725  -0.68860704   0.70062757]
 [ -8.74113075  -0.61578832   0.84174257]
 [ -7.96021974  -0.70872513   0.66059647]
 [ -9.44444444  -0.70391428   0.65253017]
 [-10.          -0.71178657   0.64045635]
 [ -8.40466419  -0.705445     0.65014642]
 [ -7.73799752  -0.79925651   0.52222816]
 [ -8.2

In [8]:

print("Scores of the primer pairs from the final Pareto front")
print(res.F)

print("Scores of the primer pairs from the final population")
print(res.pop.get("F"))

# import pandas as pd 
# pd.DataFrame(res_1_gen.pop.get("F")).to_csv("F_gen_1.csv", index=False, header=False)

res_pop_f = res.pop.get("F")

Scores of the primer pairs from the final Pareto front
[[-7.73799752 -0.79925651  0.52222816]]
Scores of the primer pairs from the final population
[[ -8.36330772  -0.78504264   0.55892714]
 [-10.          -0.71178657   0.64045635]
 [ -7.73799752  -0.79925651   0.52222816]
 [-10.          -0.71178657   0.64096161]
 [-10.          -0.71178657   0.64096161]
 [ -7.66666667  -0.79925651   0.52281025]
 [-10.          -0.69975946   0.6718967 ]
 [ -7.96021974  -0.70872513   0.66059647]
 [ -9.44444444  -0.70391428   0.65253017]
 [ -8.40466419  -0.705445     0.65014642]
 [ -9.66666667  -0.68510824   0.69536253]
 [ -7.88888889  -0.70872513   0.66103732]
 [ -9.44444444  -0.70391428   0.65295275]
 [ -8.33333333  -0.705445     0.65056872]
 [ -8.33333333  -0.705445     0.65056872]
 [ -9.07441883  -0.68663897   0.69297983]
 [ -8.96078431  -0.70129018   0.67984835]
 [ -8.6410855   -0.68991909   0.70355759]
 [ -8.63333333  -0.69188716   0.71107467]
 [ -8.8627451   -0.68707632   0.70310844]
 [ -8.843137

In the first generation, **only one solution appears to be non-dominated** but if the rest of the initial population is inspected we can see that the second solution is clearly non-dominated because it has 10 of efficiency and for this reason, it should be also counted as a non-dominated solution.

If we run the non dominated sorting implemented in pymoo, we obtain **3 non-dominated solutions** in the scores of the initial population that belong to the first Pareto front. So if these 3 solutions are non-dominated, why in the pymoo verbose mode only appear to be one non-dominated solution?

In [9]:
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting

nds = NonDominatedSorting().do(res_pop_f, only_non_dominated_front=True) # method="fast_non_dominated_sort"
print(nds)

# Non-dominated solution scores
print(res_pop_f[nds])

[0 1 2]
[[ -8.36330772  -0.78504264   0.55892714]
 [-10.          -0.71178657   0.64045635]
 [ -7.73799752  -0.79925651   0.52222816]]


After debugging the code to find out where the other two solutions are lost I saw that this happens after the non-dominated sorting and niching of the individuals in NSGA3.py script.

Some of the non-dominated solutions are discarded in this step:

In [10]:
# closest = np.unique(dist_matrix[:, np.unique(niche_of_individuals)].argmin(axis=0))
# self.opt = pop[intersect(fronts[0], closest)]

This issue of non-dominated solutions discarding occurs in all generations and I don't understand why this happens if, based on the NSGA3 reference paper, once the non-dominated sorting is done no non-dominated solutions are discarded from the fronts **before the splitting front**. For this reason, I think that if after the non-dominated sorting there are 3 non-dominated solutions in the first front, in the end, there should also be 3 and not only 1 as the verbose mode of the algorithm shows.

# Algorithm run with 10 generations

We can see how in each generation some of the non-dominated solutions are discarded.

In [11]:
termination = ('n_gen', 10)

# Optimization
res_10_gen = minimize(
    problem = problem,
    algorithm = algorithm,
    termination = termination,
    seed = pars.seed,
    verbose = True,
    save_history = False
)


Scores of the population: 
[[ -7.21083798  -0.30701946   1.50485124]
 [ -7.39394555  -0.36474962   1.36505336]
 [ -5.83274587  -0.31423573   1.51681093]
 [ -5.87688148  -0.31379838   1.51456411]
 [ -7.13539938  -0.31226766   1.48646954]
 [ -7.98785679  -0.36868576   1.35386463]
 [ -6.17112168  -0.321452     1.49954154]
 [ -8.6410855   -0.68991909   0.70355759]
 [ -9.66666667  -0.68510824   0.69536253]
 [-10.          -0.69975946   0.6718967 ]
 [ -9.07441883  -0.68663897   0.69297983]
 [ -8.36330772  -0.78504264   0.55892714]
 [ -8.4361602   -0.61403892   0.83489132]
 [ -8.63333333  -0.69188716   0.71107467]
 [ -8.8627451   -0.68707632   0.70310844]
 [ -8.96078431  -0.70129018   0.67984835]
 [ -8.84313725  -0.68860704   0.70062757]
 [ -8.74113075  -0.61578832   0.84174257]
 [ -7.96021974  -0.70872513   0.66059647]
 [ -9.44444444  -0.70391428   0.65253017]
 [-10.          -0.71178657   0.64045635]
 [ -8.40466419  -0.705445     0.65014642]
 [ -7.73799752  -0.79925651   0.52222816]
 [ -8.2

In [12]:
# Results
print("Scores of the primer pairs from the final Pareto front")
print(res_10_gen.F)

print("Scores of the primer pairs from the final population")
print(res_10_gen.pop.get("F"))
    

Scores of the primer pairs from the final Pareto front
[[-10.          -0.71353597   0.63816192]
 [ -8.66666667  -0.74152635   0.59815813]
 [ -9.8         -0.72403236   0.63563087]
 [ -8.38876795  -0.82899628   0.5089709 ]
 [ -9.33333333  -0.7375902    0.60422614]]
Scores of the primer pairs from the final population
[[ -9.6423141   -0.72381369   0.63558658]
 [-10.          -0.71353597   0.63816192]
 [ -8.66666667  -0.74152635   0.59815813]
 [ -9.60554875  -0.73146731   0.61100474]
 [ -8.43835962  -0.79007216   0.5662377 ]
 [ -8.44664105  -0.78351192   0.56312868]
 [ -9.8         -0.72403236   0.63563087]
 [ -8.38876795  -0.82899628   0.5089709 ]
 [ -8.42210129  -0.82855893   0.53469542]
 [ -8.52169295  -0.79007216   0.56699345]
 [ -8.52210129  -0.81653182   0.59551615]
 [ -9.33333333  -0.7375902    0.60422614]
 [ -8.54947073  -0.79007216   0.5852972 ]
 [-10.          -0.71266127   0.63951944]
 [-10.          -0.71266127   0.63951944]
 [-10.          -0.71266127   0.63951944]
 [ -9.444